In [ ]:
print("Hello, World!")

In [ ]:
! pip install ollama
! pip install pandas
! pip install matplotlib
! ollama pull gemma3:1b 

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='gemma3:1b', messages=[
  {
    'role': 'user',
    'content': 'In which country is the city of Bern located?',
  },
])
print(response['message']['content'])


In [ ]:
!pip install llama-index llama-index-llms-ollama llama-index-embeddings-ollama pymupdf

In [ ]:

# Load a PDF from ./resources, embed it with Ollama's embeddinggemma, 
# and query with Ollama's gemma3:1b.

import glob
from pathlib import Path

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# --- 1) Locate the PDF ---
resources_dir = Path("./resources")
pdf_paths = sorted(glob.glob(str(resources_dir / "*.pdf")))
if not pdf_paths:
    raise FileNotFoundError("No PDF found in ./input. Please place one there.")

# --- 2) Configure Ollama LLM and embedding model ---
Settings.llm = Ollama(model="gemma3:1b", request_timeout=120.0)
Settings.embed_model = OllamaEmbedding(model_name="embeddinggemma")

# --- 3) Load and index the PDF with embeddings ---
docs = SimpleDirectoryReader(input_files=[pdf_paths[0]]).load_data()
index = VectorStoreIndex.from_documents(docs)

# --- 4) Run a query ---
query_engine = index.as_query_engine(similarity_top_k=3)
prompt = "Build a slide for my presentation summarizing this document with a title and 3 bullet points."
response = query_engine.query(prompt)

print(f"Queried file: {Path(pdf_paths[0]).name}")
print("\n=== Response ===\n")
print(str(response))

In [ ]:
!pip install ollama pillow

In [ ]:
# Describe an image with Gemma3:4b via Ollama


import ollama
from PIL import Image

# --- 1) Path to your image ---
image_path = "./input/rag_simple.png"  # <-- replace with your image

# Optional: preview image in the notebook
display(Image.open(image_path))

# --- 2) Send prompt + image to Ollama ---
response = ollama.chat(
    model="gemma3:4b",
    messages=[
        {
            "role": "user",
            "content": "Build a slide for my presentation summarizing this image with a title and 3 bullet points.",
            "images": [image_path],  # send the image to the model
        }
    ],
)

# --- 3) Print the description ---
print("=== Image Description ===")
print(response["message"]["content"])